# 🎚 MAEVN - Google Colab Build & Run

This notebook allows you to build, install, and run MAEVN (AI-Powered Vocal + Instrument Generator) on Google Colab.

## Features
- ✅ **Build from Source**: Complete build environment setup
- ✅ **ONNX Models**: Export and manage AI models
- ✅ **Google Drive Integration**: Import/Export models and configuration
- ✅ **Standalone Mode**: Run MAEVN in your browser (with audio support)
- ✅ **Session Persistence**: Save/restore your setup across sessions

## Quick Start
1. Run cells in order from top to bottom
2. Mount Google Drive when prompted (optional but recommended)
3. Wait for build to complete (~10-15 minutes)
4. Launch standalone application

## Requirements
- Google Account (for Colab)
- Google Drive (optional, for model persistence)
- ~30 minutes for first-time setup

---

## 📦 Step 1: Environment Setup

Install system dependencies required for building MAEVN.

In [ ]:
%%bash
# Update package list
echo "Updating package list..."
sudo apt-get update -qq

# Install build dependencies
echo "Installing build dependencies..."
sudo apt-get install -y -qq \
    build-essential \
    cmake \
    git \
    libasound2-dev \
    libjack-jackd2-dev \
    libcurl4-openssl-dev \
    libfreetype6-dev \
    libx11-dev \
    libxcomposite-dev \
    libxcursor-dev \
    libxinerama-dev \
    libxrandr-dev \
    libxrender-dev \
    libwebkit2gtk-4.0-dev \
    libglu1-mesa-dev \
    mesa-common-dev \
    pkg-config

echo "✓ Dependencies installed successfully!"

### Verify Installation

In [ ]:
%%bash
echo "Checking installed versions..."
echo "================================"
echo "CMake: $(cmake --version | head -n1)"
echo "GCC: $(gcc --version | head -n1)"
echo "G++: $(g++ --version | head -n1)"
echo "Python: $(python3 --version)"
echo "Git: $(git --version)"
echo "================================"
echo "✓ All tools verified!"

## 💾 Step 2: Google Drive Integration (Optional)

Mount Google Drive to save/load models and configurations between sessions.

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Create MAEVN directory in Google Drive if it doesn't exist
maevn_drive_dir = '/content/drive/MyDrive/MAEVN'
os.makedirs(maevn_drive_dir, exist_ok=True)
os.makedirs(f'{maevn_drive_dir}/Models', exist_ok=True)
os.makedirs(f'{maevn_drive_dir}/Models/drums', exist_ok=True)
os.makedirs(f'{maevn_drive_dir}/Models/instruments', exist_ok=True)
os.makedirs(f'{maevn_drive_dir}/Models/vocals', exist_ok=True)

print("✓ Google Drive mounted successfully!")
print(f"✓ MAEVN directory: {maevn_drive_dir}")

## 📥 Step 3: Clone Repository

Clone the MAEVN repository from GitHub.

In [ ]:
%%bash
# Change to content directory
cd /content

# Remove existing clone if present
if [ -d "Voice_Clone-VST" ]; then
    echo "Removing existing repository..."
    rm -rf Voice_Clone-VST
fi

# Clone repository
echo "Cloning MAEVN repository..."
git clone https://github.com/fxgeniusllc-oss/Voice_Clone-VST.git

echo "✓ Repository cloned successfully!"

## 🔧 Step 4: Repository Setup

Setup the repository directory structure.

In [ ]:
%%bash
cd /content/Voice_Clone-VST

# Run repository setup script
echo "Setting up repository..."
chmod +x setup_maevn_repo.sh
./setup_maevn_repo.sh

echo "✓ Repository setup complete!"

## 📦 Step 5: Restore Models from Google Drive (Optional)

If you have previously exported models to Google Drive, restore them here.

In [ ]:
import os
import shutil

maevn_drive_dir = '/content/drive/MyDrive/MAEVN'
local_models_dir = '/content/Voice_Clone-VST/Models'

if os.path.exists(maevn_drive_dir):
    print("Checking for existing models in Google Drive...")
    
    # Check for existing models
    drive_models = f'{maevn_drive_dir}/Models'
    if os.path.exists(drive_models) and os.listdir(drive_models):
        print(f"Found models in Google Drive. Copying to local repository...")
        
        # Copy models from Drive to local
        for item in ['drums', 'instruments', 'vocals']:
            src = f'{drive_models}/{item}'
            dst = f'{local_models_dir}/{item}'
            if os.path.exists(src):
                # Copy .onnx files
                for file in os.listdir(src):
                    if file.endswith('.onnx'):
                        shutil.copy2(f'{src}/{file}', f'{dst}/{file}')
                        print(f"  ✓ Restored {item}/{file}")
        
        # Copy config files if they exist
        for config_file in ['config.json', 'metadata.json']:
            src_config = f'{drive_models}/{config_file}'
            dst_config = f'{local_models_dir}/{config_file}'
            if os.path.exists(src_config):
                shutil.copy2(src_config, dst_config)
                print(f"  ✓ Restored {config_file}")
        
        print("✓ Models restored from Google Drive!")
    else:
        print("No existing models found in Google Drive. Will export new ones.")
else:
    print("Google Drive not mounted or MAEVN directory not found.")
    print("Skipping model restoration.")

## 🧠 Step 6: Export ONNX Models

Export default ONNX models for AI features. Skip this if you restored models from Google Drive.

In [ ]:
# Install Python dependencies for model export
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip install -q onnx numpy

print("✓ Python dependencies installed!")

In [ ]:
%%bash
cd /content/Voice_Clone-VST

# Make script executable
chmod +x build_maevn_onnx.sh

# Export models
echo "Exporting ONNX models..."
./build_maevn_onnx.sh

echo "✓ ONNX models exported successfully!"

## 💾 Step 7: Backup Models to Google Drive (Optional)

Save exported models to Google Drive for reuse in future sessions.

In [ ]:
import os
import shutil

maevn_drive_dir = '/content/drive/MyDrive/MAEVN'
local_models_dir = '/content/Voice_Clone-VST/Models'

if os.path.exists('/content/drive/MyDrive'):
    print("Backing up models to Google Drive...")
    
    # Ensure drive directories exist
    os.makedirs(f'{maevn_drive_dir}/Models/drums', exist_ok=True)
    os.makedirs(f'{maevn_drive_dir}/Models/instruments', exist_ok=True)
    os.makedirs(f'{maevn_drive_dir}/Models/vocals', exist_ok=True)
    
    # Copy model files
    for item in ['drums', 'instruments', 'vocals']:
        src = f'{local_models_dir}/{item}'
        dst = f'{maevn_drive_dir}/Models/{item}'
        if os.path.exists(src):
            for file in os.listdir(src):
                if file.endswith('.onnx'):
                    shutil.copy2(f'{src}/{file}', f'{dst}/{file}')
                    print(f"  ✓ Backed up {item}/{file}")
    
    # Copy config files
    for config_file in ['config.json', 'metadata.json']:
        src_config = f'{local_models_dir}/{config_file}'
        if os.path.exists(src_config):
            shutil.copy2(src_config, f'{maevn_drive_dir}/Models/{config_file}')
            print(f"  ✓ Backed up {config_file}")
    
    print("✓ Models backed up to Google Drive!")
    print(f"Location: {maevn_drive_dir}/Models")
else:
    print("Google Drive not mounted. Skipping backup.")

## 🔨 Step 8: Build MAEVN

Configure and build the MAEVN plugin using CMake. This may take 10-15 minutes.

In [ ]:
%%bash
cd /content/Voice_Clone-VST

# Create build directory
echo "Creating build directory..."
rm -rf build
mkdir build
cd build

# Configure with CMake
echo "Configuring with CMake..."
cmake .. -DCMAKE_BUILD_TYPE=Release

echo "✓ CMake configuration complete!"

In [ ]:
%%bash
cd /content/Voice_Clone-VST/build

# Build the plugin (this takes a while)
echo "Building MAEVN... (this may take 10-15 minutes)"
cmake --build . --config Release -j$(nproc)

echo "✓ Build complete!"

## ✅ Step 9: Verify Build

Check that the build artifacts were created successfully.

In [ ]:
import os

build_dir = '/content/Voice_Clone-VST/build'
artefacts_dir = f'{build_dir}/MAEVN_artefacts/Release'

print("Checking build artifacts...")
print("="*50)

# Check for Standalone
standalone = f'{artefacts_dir}/Standalone/MAEVN'
if os.path.exists(standalone):
    print("✓ Standalone executable found")
    print(f"  Location: {standalone}")
else:
    print("✗ Standalone executable NOT found")

# Check for VST3
vst3 = f'{artefacts_dir}/VST3/MAEVN.vst3'
if os.path.exists(vst3):
    print("✓ VST3 plugin found")
    print(f"  Location: {vst3}")
else:
    print("✗ VST3 plugin NOT found")

print("="*50)

if os.path.exists(standalone):
    print("\n✓ Build verification successful!")
else:
    print("\n✗ Build verification failed. Check the build output above.")

## 📦 Step 10: Install MAEVN

Install the standalone application and VST3 plugin.

In [ ]:
%%bash
cd /content/Voice_Clone-VST

# Make install script executable
chmod +x install_maevn.sh

# Run installation (as non-root for user installation)
echo "Installing MAEVN..."
./install_maevn.sh

echo "✓ Installation complete!"

## 🚀 Step 11: Launch MAEVN Standalone

### Important Notes for Google Colab:

**Audio Support:** Google Colab does not provide direct audio output. However, you can:
1. Run MAEVN to test the build
2. Use MAEVN to generate audio files
3. Download the VST3 plugin to use on your local machine

**GUI Support:** Colab doesn't support X11 forwarding by default, so the GUI may not display.

### For actual usage:
Download the VST3 plugin or standalone executable to your local machine.

In [ ]:
%%bash
cd /content/Voice_Clone-VST

# Test that the executable exists and is valid
if [ -f "build/MAEVN_artefacts/Release/Standalone/MAEVN" ]; then
    echo "✓ MAEVN standalone executable is ready!"
    echo "  Location: build/MAEVN_artefacts/Release/Standalone/MAEVN"
    
    # Check if it's executable
    if [ -x "build/MAEVN_artefacts/Release/Standalone/MAEVN" ]; then
        echo "✓ Executable has correct permissions"
    else
        echo "✗ Making executable..."
        chmod +x build/MAEVN_artefacts/Release/Standalone/MAEVN
    fi
    
    # Get file info
    echo ""
    echo "File information:"
    ls -lh build/MAEVN_artefacts/Release/Standalone/MAEVN
    file build/MAEVN_artefacts/Release/Standalone/MAEVN
else
    echo "✗ MAEVN executable not found!"
fi

## 📥 Step 12: Download Build Artifacts

Download the VST3 plugin and standalone executable to use on your local machine.

In [ ]:
%%bash
cd /content/Voice_Clone-VST

# Create package directory
echo "Creating package..."
mkdir -p /content/MAEVN_Package

# Copy VST3 plugin
if [ -d "build/MAEVN_artefacts/Release/VST3/MAEVN.vst3" ]; then
    echo "Packaging VST3 plugin..."
    cp -r build/MAEVN_artefacts/Release/VST3/MAEVN.vst3 /content/MAEVN_Package/
fi

# Copy standalone executable
if [ -f "build/MAEVN_artefacts/Release/Standalone/MAEVN" ]; then
    echo "Packaging standalone executable..."
    cp build/MAEVN_artefacts/Release/Standalone/MAEVN /content/MAEVN_Package/
fi

# Copy models
echo "Packaging models..."
cp -r Models /content/MAEVN_Package/

# Copy documentation
echo "Packaging documentation..."
cp README.md QUICKSTART.md BUILD.md /content/MAEVN_Package/ 2>/dev/null || true

# Create archive
echo "Creating archive..."
cd /content
tar -czf MAEVN_Package.tar.gz MAEVN_Package/

echo "✓ Package created: /content/MAEVN_Package.tar.gz"

In [ ]:
from google.colab import files
import os

package_file = '/content/MAEVN_Package.tar.gz'

if os.path.exists(package_file):
    print("Downloading MAEVN package...")
    print(f"Package size: {os.path.getsize(package_file) / (1024*1024):.2f} MB")
    files.download(package_file)
    print("✓ Download started!")
else:
    print("✗ Package file not found!")

## 💾 Step 13: Backup Package to Google Drive (Optional)

Save the complete package to Google Drive for easy access.

In [ ]:
import shutil
import os

package_file = '/content/MAEVN_Package.tar.gz'
drive_backup = '/content/drive/MyDrive/MAEVN/MAEVN_Package.tar.gz'

if os.path.exists('/content/drive/MyDrive'):
    if os.path.exists(package_file):
        print("Backing up package to Google Drive...")
        shutil.copy2(package_file, drive_backup)
        print(f"✓ Package backed up to: {drive_backup}")
        print(f"Size: {os.path.getsize(drive_backup) / (1024*1024):.2f} MB")
    else:
        print("✗ Package file not found!")
else:
    print("Google Drive not mounted. Skipping backup.")

## 📖 Usage Instructions

### On Your Local Machine:

1. **Extract the package:**
   ```bash
   tar -xzf MAEVN_Package.tar.gz
   cd MAEVN_Package
   ```

2. **Install VST3 Plugin:**
   - **Windows:** Copy `MAEVN.vst3` to `C:\Program Files\Common Files\VST3\`
   - **macOS:** Copy `MAEVN.vst3` to `~/Library/Audio/Plug-Ins/VST3/`
   - **Linux:** Copy `MAEVN.vst3` to `~/.vst3/`

3. **Run Standalone:**
   - **Linux:** `./MAEVN`
   - **macOS/Windows:** Build required on local machine (use plugin instead)

4. **Rescan plugins in your DAW** and load MAEVN as an instrument

### Compatible DAWs:
- Ableton Live 10+
- FL Studio 20+
- Reaper 5.0+
- Bitwig Studio 3.0+
- Steinberg Cubase 10.5+
- PreSonus Studio One 4+

### Documentation:
- See `README.md` for overview
- See `QUICKSTART.md` for usage guide
- See `BUILD.md` for build details

---

## 🔧 Troubleshooting

### Build Fails
- Re-run the "Environment Setup" cell
- Check that CMake version is 3.15+
- Clear build directory and retry: `rm -rf /content/Voice_Clone-VST/build`

### Models Not Found
- Re-run "Export ONNX Models" cell
- Check that Python dependencies are installed
- Verify PyTorch installation: `python3 -c "import torch; print(torch.__version__)"`

### Google Drive Issues
- Ensure you granted permission when mounting Drive
- Check that MAEVN folder exists in Drive
- Remount Drive and retry

### Package Download Fails
- Check package size is reasonable (<500MB)
- Try downloading from Google Drive instead
- Use the backup in Drive: `/content/drive/MyDrive/MAEVN/MAEVN_Package.tar.gz`

### Out of Memory
- Google Colab has limited RAM
- Close other notebooks
- Restart runtime and try again
- Use Colab Pro for more resources

---

## 🧹 Cleanup (Optional)

Free up space by removing build artifacts after downloading.

In [ ]:
%%bash
echo "Cleaning up build artifacts..."

# Remove build directory
rm -rf /content/Voice_Clone-VST/build

# Remove package directory
rm -rf /content/MAEVN_Package

# Keep package archive and repository
echo "✓ Cleanup complete!"
echo "Kept: /content/MAEVN_Package.tar.gz"
echo "Kept: /content/Voice_Clone-VST (source code)"

---

## 📚 Additional Resources

- **GitHub Repository:** https://github.com/fxgeniusllc-oss/Voice_Clone-VST
- **Documentation:** See packaged README files
- **Issues/Support:** https://github.com/fxgeniusllc-oss/Voice_Clone-VST/issues

## 🌟 Next Steps

1. Download the package to your local machine
2. Install the VST3 plugin in your DAW
3. Read QUICKSTART.md for usage instructions
4. Experiment with MAEVN's features
5. Share your creations!

---

**Built with ❤️ by the MAEVN Team**

License: MIT
